# Time Series with Quantity Invoiced Data Sliding Window

## Import Libraries and Loading Data

In [1]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Analysis run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 3, Finished, Available, Finished)

Libraries imported successfully!
Analysis run on: 2025-07-24 22:20:43


In [2]:
# Check whether running in Fabric or locally, and set the data location accordingly
if "AZURE_SERVICE" in os.environ:
    is_fabric = True
    data_location = "abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db/Files/forecasting/"
    print("Running in Fabric, setting data location to /lakehouse/default/Files/")
else:
    is_fabric = False
    data_location = ""
    print("Running locally, setting data location to current directory")

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 4, Finished, Available, Finished)

Running in Fabric, setting data location to /lakehouse/default/Files/


In [3]:
# Load the combined sales economic data
data = pd.read_csv(data_location + 'modelGeneratedData/overall_monthly_with_economic_and_future.csv')
# data = pd.read_csv(data_location + 'modelGeneratedData/overall_monthly_with_trend_seasonality_economic_and_future.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

print("=== DATASET OVERVIEW ===")
print(f"Dataset shape: {data.shape}")
print(f"Date range: {data['Date'].min()} to {data['Date'].max()}")
print(f"\nData types:")
print(data.dtypes)
print(f"\nFirst few rows:")
data.head()

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 5, Finished, Available, Finished)

=== DATASET OVERVIEW ===
Dataset shape: (121, 57)
Date range: 2015-01-01 00:00:00 to 2025-01-01 00:00:00

Data types:
Date                                  datetime64[ns]
Quantity Invoiced                              int64
Quantity Invoiced Mean                       float64
Quantity Invoiced Count                        int64
Unique Customers                               int64
Unique Products                                int64
Unique Categories                              int64
Unique SubCategories                           int64
Unique EndMarkets L1                           int64
Unique EndMarkets L2                           int64
future_orders_count                            int64
future_orders_qty_total                        int64
future_orders_qty_next_1m                      int64
future_orders_qty_next_3m                      int64
future_orders_qty_next_6m                      int64
future_orders_qty_next_12m                     int64
future_orders_avg_lead_time       

,Date,Quantity Invoiced,Quantity Invoiced Mean,Quantity Invoiced Count,Unique Customers,Unique Products,Unique Categories,Unique SubCategories,Unique EndMarkets L1,Unique EndMarkets L2,...,data_Factory_Utilization,data_Capacity_Utilization,Electricity Price,Electricity Price (Lag6),Gas Price,Gas Price (Lag6),Global Supply Chain Pressure Index,GSCPI (Lag1),Manufacturing Orders Volume Index,MOVI (Lag6)
0,2015-01-01,22725114,68039.26347,334,160,53,4,15,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.50,-0.39,86.8,76.1
1,2015-02-01,23032809,63276.94780,364,183,52,4,15,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.32,-0.50,98.8,89.3
2,2015-03-01,27527951,76679.52925,359,180,50,4,12,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.38,-0.32,90.0,91.5
3,2015-04-01,25864804,68789.37234,376,177,58,4,15,3,6,...,0.4372,76.8014,0.2276,0.2302,14.46,15.23,-0.35,-0.38,83.3,87.4
4,2015-05-01,22517479,77379.65292,291,146,51,4,14,3,5,...,0.4381,76.8657,0.2276,0.2302,14.46,15.23,-0.54,-0.35,98.0,88.1


## Data Preparation for Overall Forecasting

We'll prepare the data for overall quantity forecasting by aggregating all sales data across segments and subcategories to create a single time series for the total quantity.

In [4]:
# Create overall aggregation
print("=== CREATING OVERALL AGGREGATION ===")

# Print column names
print(f"Columns in dataset: {data.columns.tolist()}")

# Rename columns in the data
data = data.rename(columns={
    'data_PP_Spot': 'PP_Spot',
    'data_Resin': 'Resin',
    'data_WTI_Crude_Oil': 'WTI_Crude_Oil',
    'data_Natural_Gas': 'Natural_Gas',
    'data_Energy_Average': 'Energy_Average',
    'data_PPI_Freight': 'PPI_Freight',
    'data_PMI_Data': 'PMI_Data',
    'data_Factory_Utilization': 'Factory_Utilization',
    'data_Capacity_Utilization': 'Capacity_Utilization',
    'data_Beverage': 'Beverage',
    'data_Household_consumption': 'Household_consumption',
    'data_packaging': 'packaging',
    'data_Diesel': 'Diesel',
    'data_PPI_Delivery': 'PPI_Delivery',
    'data_Oil-to-resin': 'Oil-to-resin',
    'Electricity Price': 'Electricity Price',
    'Electricity Price (Lag6)': 'Electricity Price (Lag6)',
    'Gas Price': 'Gas Price',
    'Gas Price (Lag6)': 'Gas Price (Lag6)',
    'Global Supply Chain Pressure Index': 'Global Supply Chain Pressure Index',
    'GSCPI (Lag1)': 'GSCPI (Lag1)',
    'Manufacturing Orders Volume Index': 'Manufacturing Orders Volume Index',
    'MOVI (Lag6)': 'MOVI (Lag6)',
    'packaging (Lag2)': 'packaging (Lag2)',
    'PPI_Freight (Lag2)': 'PPI_Freight (Lag2)',
    'trend': 'decomp_trend',
    'seasonality': 'decomp_seasonality',
})

# Define exogenous variables for modeling
exog_vars = [
    'PP_Spot',
    'Resin',
    'PMI_Data',
    'Natural_Gas',
    # 'WTI_Crude_Oil',
    #'Factory_Utilization',
    'packaging',
    'Energy_Average',
    'Electricity Price (Lag6)',
    'Gas Price (Lag6)',
    'Global Supply Chain Pressure Index', # neutral
    # 'future_orders_qty_total',
    'future_orders_qty_next_1m',
    'future_orders_qty_next_3m',
    #'future_orders_qty_next_6m',
    # 'future_orders_qty_next_12m',
    # 'future_orders_avg_lead_time',
    # 'future_orders_min_lead_time',
    # 'future_orders_max_lead_time',
    # 'decomp_trend', 
    #'decomp_seasonality'
]

# Overall aggregation (sum across all segments and subcategories)
print(f"Overall time series shape: {data.shape}")

# Display summary statistics
print("\n=== OVERALL SUMMARY ===")
print(f"Overall total quantity range: {data['Quantity Invoiced'].min():,.0f} - {data['Quantity Invoiced'].max():,.0f}")
print(f"Date range: {data['Date'].min()} to {data['Date'].max()}")
print(f"Number of data points: {len(data)}")

# Display columns and their data types
print("\n=== COLUMNS AND DATA TYPES ===")
print(data.dtypes)

data.head()

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 6, Finished, Available, Finished)

=== CREATING OVERALL AGGREGATION ===
Columns in dataset: ['Date', 'Quantity Invoiced', 'Quantity Invoiced Mean', 'Quantity Invoiced Count', 'Unique Customers', 'Unique Products', 'Unique Categories', 'Unique SubCategories', 'Unique EndMarkets L1', 'Unique EndMarkets L2', 'future_orders_count', 'future_orders_qty_total', 'future_orders_qty_next_1m', 'future_orders_qty_next_3m', 'future_orders_qty_next_6m', 'future_orders_qty_next_12m', 'future_orders_avg_lead_time', 'future_orders_min_lead_time', 'future_orders_max_lead_time', 'future_orders_due_next_month', 'future_orders_due_next_quarter', 'future_orders_unique_customers', 'future_orders_unique_products', 'future_to_current_ratio', 'qty_rolling_avg_3m', 'future_to_rolling_3m_ratio', 'qty_rolling_avg_12m', 'future_to_rolling_12m_ratio', 'has_future_orders', 'high_future_orders', 'future_customer_diversity_ratio', 'future_product_diversity_ratio', 'data_PP_Spot', 'data_Resin', 'data_WTI_Crude_Oil', 'data_Natural_Gas', 'data_Oil-to-resin

,Date,Quantity Invoiced,Quantity Invoiced Mean,Quantity Invoiced Count,Unique Customers,Unique Products,Unique Categories,Unique SubCategories,Unique EndMarkets L1,Unique EndMarkets L2,...,Factory_Utilization,Capacity_Utilization,Electricity Price,Electricity Price (Lag6),Gas Price,Gas Price (Lag6),Global Supply Chain Pressure Index,GSCPI (Lag1),Manufacturing Orders Volume Index,MOVI (Lag6)
0,2015-01-01,22725114,68039.26347,334,160,53,4,15,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.50,-0.39,86.8,76.1
1,2015-02-01,23032809,63276.94780,364,183,52,4,15,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.32,-0.50,98.8,89.3
2,2015-03-01,27527951,76679.52925,359,180,50,4,12,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.38,-0.32,90.0,91.5
3,2015-04-01,25864804,68789.37234,376,177,58,4,15,3,6,...,0.4372,76.8014,0.2276,0.2302,14.46,15.23,-0.35,-0.38,83.3,87.4
4,2015-05-01,22517479,77379.65292,291,146,51,4,14,3,5,...,0.4381,76.8657,0.2276,0.2302,14.46,15.23,-0.54,-0.35,98.0,88.1


## Split Data into Train and Test Pairs

In [5]:
def sliding_window(data, window_size_months=108, horizon=3):
    """Generates sliding-window train-test splits with fixed train size"""
    train_test_splits = []
    
    # start_date = data['Date'].min()
    start_date = pd.to_datetime('2015-01-01')
    max_date = data['Date'].max()
    
    # while start_date + pd.DateOffset(months=window_size_months + horizon) <= max_date + pd.DateOffset(months=1) - pd.DateOffset(months=0):
    while start_date + pd.DateOffset(months=window_size_months + horizon) <= max_date + pd.DateOffset(months=1) - pd.DateOffset(months=1):  # exclude 2025-01-01 data
        train_start = start_date
        print(f"Train Start: {train_start.strftime('%Y-%m')}")
        train_end = train_start + pd.DateOffset(months=window_size_months - 1)
        print(f"Train End  : {train_end.strftime('%Y-%m')}")
        test_start = train_end + pd.DateOffset(months=1)
        print(f"Test Start : {test_start.strftime('%Y-%m')}")
        test_end = test_start + pd.DateOffset(months=horizon - 1)
        print(f"Test End   : {test_end.strftime('%Y-%m')}")

        train_data = data[(data['Date'] >= train_start) & (data['Date'] <= train_end)]
        test_data = data[(data['Date'] >= test_start) & (data['Date'] <= test_end)]
        
        train_test_splits.append((train_data.copy(), test_data.copy()))
        
        # Slide the window forward by 1 month
        start_date += pd.DateOffset(months=1)

    return train_test_splits


StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 7, Finished, Available, Finished)

In [6]:
# Use a fixed training window
sliding_splits = sliding_window(data, window_size_months=108, horizon=3)
print(f"Generated {len(sliding_splits)} sliding-window splits.")


StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 8, Finished, Available, Finished)

Train Start: 2015-01
Train End  : 2023-12
Test Start : 2024-01
Test End   : 2024-03
Train Start: 2015-02
Train End  : 2024-01
Test Start : 2024-02
Test End   : 2024-04
Train Start: 2015-03
Train End  : 2024-02
Test Start : 2024-03
Test End   : 2024-05
Train Start: 2015-04
Train End  : 2024-03
Test Start : 2024-04
Test End   : 2024-06
Train Start: 2015-05
Train End  : 2024-04
Test Start : 2024-05
Test End   : 2024-07
Train Start: 2015-06
Train End  : 2024-05
Test Start : 2024-06
Test End   : 2024-08
Train Start: 2015-07
Train End  : 2024-06
Test Start : 2024-07
Test End   : 2024-09
Train Start: 2015-08
Train End  : 2024-07
Test Start : 2024-08
Test End   : 2024-10
Train Start: 2015-09
Train End  : 2024-08
Test Start : 2024-09
Test End   : 2024-11
Train Start: 2015-10
Train End  : 2024-09
Test Start : 2024-10
Test End   : 2024-12
Generated 10 sliding-window splits.


In [7]:
train_df, test_df = sliding_splits[0]

print("📊 Train Data:")
display(train_df.head())

print("📈 Test Data:")
display(test_df.head())


StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 9, Finished, Available, Finished)

📊 Train Data:


SynapseWidget(Synapse.DataFrame, 90a52269-a070-4e7b-8b7d-8ba111fd78b6)

📈 Test Data:


SynapseWidget(Synapse.DataFrame, 45366e3b-9adb-4790-a86d-ec091f553920)

# Overall Quantity Forecasting Implementation

Functions defining the forecasting methods for overall quantity prediction.

## Forecasting Models

We'll implement multiple forecasting approaches:
1. **ARIMA** - Auto-regressive Integrated Moving Average
2. **SARIMA** - Seasonal ARIMA with economic indicators
3. **Exponential Smoothing** - Holt-Winters method
4. **Prophet** - Meta's time series forecasting tool with trend and seasonality
5. **Extra Trees** - Ensemble machine learning method using sliding window features
6. **Ensemble** - Weighted combination of methods

In [8]:
def forecast_arima(series, steps=12, order=(1,1,1)):
    """
    ARIMA forecasting with automatic order selection if needed
    """
    try:
        model = ARIMA(series, order=order)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=steps)
        conf_int = fitted_model.get_forecast(steps=steps).conf_int()
        return forecast, conf_int, fitted_model.aic
    except:
        # Try simpler model if original fails
        try:
            model = ARIMA(series, order=(1,0,1))
            fitted_model = model.fit()
            forecast = fitted_model.forecast(steps=steps)
            conf_int = fitted_model.get_forecast(steps=steps).conf_int()
            return forecast, conf_int, fitted_model.aic
        except:
            # Last resort - simple naive forecast
            last_value = series.iloc[-1]
            forecast = pd.Series([last_value] * steps)
            conf_int = pd.DataFrame({
                'lower Quantity Invoiced': forecast * 0.9,
                'upper Quantity Invoiced': forecast * 1.1
            })
            return forecast, conf_int, float('inf')

def forecast_sarima(series, steps=12, exog=None, order=(1,1,1), seasonal_order=(1,1,1,12)):
    """
    SARIMA forecasting with external regressors
    """
    try:
        model = SARIMAX(series, exog=exog, order=order, seasonal_order=seasonal_order)
        fitted_model = model.fit(disp=False)
        
        # For forecast, we need future exogenous variables
        # Use last known values as a simple assumption
        if exog is not None:
            future_exog = pd.DataFrame([exog.iloc[-1]] * steps)
            future_exog.index = pd.date_range(start=exog.index[-1] + pd.DateOffset(months=1), periods=steps, freq='MS')
        else:
            future_exog = None
            
        forecast = fitted_model.forecast(steps=steps, exog=future_exog)
        conf_int = fitted_model.get_forecast(steps=steps, exog=future_exog).conf_int()
        return forecast, conf_int, fitted_model.aic
    except:
        # Fallback to simple ARIMA
        return forecast_arima(series, steps, order)

def forecast_exponential_smoothing(series, steps=12, seasonal_periods=12):
    """
    Exponential Smoothing (Holt-Winters) forecasting
    """
    try:
        if len(series) >= 2 * seasonal_periods:
            model = ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
        else:
            model = ExponentialSmoothing(series, trend='add', seasonal=None)
        
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=steps)
        
        # Simple confidence intervals based on residuals
        residuals = fitted_model.resid
        std_resid = residuals.std()
        conf_int = pd.DataFrame({
            'lower Quantity Invoiced': forecast - 1.96 * std_resid,
            'upper Quantity Invoiced': forecast + 1.96 * std_resid
        })
        
        return forecast, conf_int, fitted_model.aic
    except:
        # Fallback to ARIMA
        return forecast_arima(series, steps)

def forecast_prophet(series, steps=12, exog=None):
    """
    Prophet forecasting with trend, seasonality, and external regressors
    """
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    prophet_data = pd.DataFrame({
        'ds': series.index,
        'y': series.values
    })
    
    # Initialize Prophet model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,  # Monthly data doesn't need weekly seasonality
        daily_seasonality=False,   # Monthly data doesn't need daily seasonality
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,  # Controls flexibility of trend
        seasonality_prior_scale=10.0,  # Controls flexibility of seasonality
        interval_width=0.95
    )
    
    # Add external regressors if provided
    if exog is not None and len(exog.columns) > 0:
        # Add each regressor to the model
        for col in exog.columns:
            if col in ['PP_Spot', 'Resin', 'WTI_Crude_Oil', 'Natural_Gas', 'Energy_Average']:
                # Energy-related indicators tend to have strong impact
                model.add_regressor(col, prior_scale=0.5, mode='multiplicative')
            elif col in ['PMI_Data', 'Factory_Utilization', 'Capacity_Utilization']:
                # Manufacturing indicators
                model.add_regressor(col, prior_scale=0.3, mode='additive')
            else:
                # Other economic indicators
                model.add_regressor(col, prior_scale=0.1, mode='additive')
        
        # Add regressors to prophet_data
        for col in exog.columns:
            prophet_data[col] = exog[col].values
    
    # Fit the model
    model.fit(prophet_data)
    
    # Create future dataframe
    last_date = series.index[-1]
    future_dates = pd.date_range(
        start=last_date + pd.DateOffset(months=1), 
        periods=steps, 
        freq='MS'
    )
    
    future_df = pd.DataFrame({'ds': future_dates})
    
    # Add future regressor values if available
    if exog is not None and len(exog.columns) > 0:
        # Use last known values for future regressors (simple assumption)
        last_regressor_values = exog.iloc[-1]
        for col in exog.columns:
            future_df[col] = last_regressor_values[col]
    
    # Make forecast
    forecast_df = model.predict(future_df)
    
    # Extract forecast values and confidence intervals
    forecast = pd.Series(
        forecast_df['yhat'].values, 
        index=future_dates,
        name='Quantity Invoiced'
    )
    
    conf_int = pd.DataFrame({
        'lower Quantity Invoiced': forecast_df['yhat_lower'].values,
        'upper Quantity Invoiced': forecast_df['yhat_upper'].values
    }, index=future_dates)
    
    # Calculate approximate AIC using cross-validation or residual-based metric
    # Prophet doesn't have built-in AIC, so we'll use MAE on in-sample predictions
    in_sample_forecast = model.predict(prophet_data)
    mae = mean_absolute_error(prophet_data['y'], in_sample_forecast['yhat'])
    pseudo_aic = 2 * len(prophet_data) + 2 * np.log(mae)  # Approximation
    
    return forecast, conf_int, pseudo_aic

def forecast_extra_trees(series, steps=12, exog=None, n_estimators=100, window_size=12):
    """
    Extra Trees forecasting using sliding window approach with external regressors
    """
    # Prepare time series features using sliding window
    def create_time_series_features(ts, window_size, exog_data=None):
        X, y = [], []
        
        for i in range(window_size, len(ts)):
            # Lagged values as features
            lag_features = ts.iloc[i-window_size:i].values
            
            # Add time-based features
            date_idx = ts.index[i]
            time_features = [
                date_idx.month,           # Month of year
                date_idx.quarter,         # Quarter
                date_idx.dayofyear / 365, # Day of year normalized
                np.sin(2 * np.pi * date_idx.month / 12),  # Seasonal sine
                np.cos(2 * np.pi * date_idx.month / 12),  # Seasonal cosine
            ]
            
            # Combine features
            features = list(lag_features) + time_features
            
            # Add exogenous variables if provided
            if exog_data is not None and i < len(exog_data):
                exog_features = exog_data.iloc[i].values.tolist()
                features.extend(exog_features)
            
            X.append(features)
            y.append(ts.iloc[i])
        
        return np.array(X), np.array(y)
    
    # Create training features
    X_train, y_train = create_time_series_features(series, window_size, exog)
    
    if len(X_train) == 0:
        raise ValueError("Not enough data for Extra Trees forecasting")
    
    # Train Extra Trees model
    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        random_state=42,
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        bootstrap=True,
        n_jobs=-1
    )
    
    model.fit(X_train, y_train)
    
    # Generate forecasts step by step
    forecast_values = []
    current_series = series.copy()
    
    for step in range(steps):
        # Prepare features for next prediction
        if len(current_series) >= window_size:
            lag_features = current_series.iloc[-window_size:].values
        else:
            # Pad with the last available values if series is too short
            pad_length = window_size - len(current_series)
            padding = [current_series.iloc[-1]] * pad_length
            lag_features = np.array(padding + current_series.tolist())
        
        # Create future date
        next_date = current_series.index[-1] + pd.DateOffset(months=1)
        
        # Time-based features for future date
        time_features = [
            next_date.month,
            next_date.quarter,
            next_date.dayofyear / 365,
            np.sin(2 * np.pi * next_date.month / 12),
            np.cos(2 * np.pi * next_date.month / 12),
        ]
        
        # Combine features
        features = list(lag_features) + time_features
        
        # Add exogenous variables (use last known values)
        if exog is not None:
            exog_features = exog.iloc[-1].values.tolist()
            features.extend(exog_features)
        
        # Make prediction
        next_pred = model.predict([features])[0]
        forecast_values.append(next_pred)
        
        # Update series with new prediction for next iteration
        current_series = pd.concat([
            current_series, 
            pd.Series([next_pred], index=[next_date])
        ])
    
    # Create forecast series with proper index
    future_dates = pd.date_range(
        start=series.index[-1] + pd.DateOffset(months=1),
        periods=steps,
        freq='MS'
    )
    
    forecast = pd.Series(forecast_values, index=future_dates, name='Quantity Invoiced')
    
    # Create confidence intervals based on model's prediction variance
    # For Extra Trees, we can use the std of predictions from individual trees
    predictions_all_trees = np.array([tree.predict([features]) for tree in model.estimators_])
    prediction_std = np.std(predictions_all_trees.flatten()) if len(predictions_all_trees) > 1 else forecast.std()
    
    conf_int = pd.DataFrame({
        'lower Quantity Invoiced': forecast - 1.96 * prediction_std,
        'upper Quantity Invoiced': forecast + 1.96 * prediction_std
    }, index=future_dates)
    
    # Calculate pseudo-AIC based on out-of-bag error
    oob_score = model.score(X_train, y_train) if hasattr(model, 'score') else 0.5
    pseudo_aic = 2 * len(X_train[0]) - 2 * np.log(max(oob_score, 0.001))
    
    return forecast, conf_int, pseudo_aic

def ensemble_forecast(forecasts, aics=None):
    """
    Create ensemble forecast from multiple methods (weighted by inverse AIC)
    """
    weights = []

    if aics is None:
        weights = [1/len(forecasts)] * len(forecasts)
    else:
        weights = [1/aic if aic != float('inf') else 0 for aic in aics]
        total_weight = sum(weights)
        if total_weight > 0:
            weights = [w/total_weight for w in weights]
        else:
            # Equal weights if all AICs are infinite
            weights = [1/len(forecasts)] * len(forecasts)

    # If forecast index is not aligned, reindex to the first forecast's index
    first_index = forecasts[0].index
    for i in range(len(forecasts)):
        if not forecasts[i].index.equals(first_index):
            forecasts[i] = forecasts[i].reindex(first_index)

    # Print weights
    print(f"Model weights - ARIMA: {weights[0]:.3f}, SARIMA: {weights[1]:.3f}, EXP: {weights[2]:.3f}, Prophet: {weights[3]:.3f}")
    
    ensemble = sum(f * w for f, w in zip(forecasts, weights))

    return ensemble

print("Forecasting functions defined successfully!")

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 10, Finished, Available, Finished)

Forecasting functions defined successfully!


In [9]:
def forecast_overall(data, exog_vars, forecast_steps, forecast_dates):
    """
    Generate overall forecasts using multiple methods and ensemble approach
    
    Parameters:
    - data: DataFrame with overall time series data
    - exog_vars: List of exogenous variables to use in forecasting
    - forecast_steps: Number of steps to forecast
    - forecast_dates: Date range for forecasts
    
    Returns:
    - DataFrame with all forecast methods and ensemble result
    """
    print("=== LEVEL 0: OVERALL FORECASTING ===")
    
    # Prepare overall data
    overall_series = data.set_index('Date')['Quantity Invoiced']
    display(data.head())
    overall_exog = data.set_index('Date')[exog_vars]
    
    # Generate forecasts using different methods
    print("Generating ARIMA forecast...")
    overall_arima_forecast, overall_arima_conf, overall_arima_aic = forecast_arima(overall_series, forecast_steps)
    
    print("Generating SARIMA forecast...")
    overall_sarima_forecast, overall_sarima_conf, overall_sarima_aic = forecast_sarima(overall_series, forecast_steps, overall_exog)
    
    print("Generating Exponential Smoothing forecast...")
    overall_exp_forecast, overall_exp_conf, overall_exp_aic = forecast_exponential_smoothing(overall_series, forecast_steps)
    
    print("Generating Prophet forecast...")
    overall_prophet_forecast, overall_prophet_conf, overall_prophet_aic = forecast_prophet(overall_series, forecast_steps, overall_exog)

    
    # Print exponential smoothing forecast
    print("Exponential Smoothing Forecast:")
    print(overall_exp_forecast.head())

    # Create ensemble forecast
    aics = [overall_arima_aic, overall_sarima_aic, overall_exp_aic, overall_prophet_aic]    
    overall_ensemble_forecast = ensemble_forecast(
        [overall_arima_forecast, overall_sarima_forecast, overall_exp_forecast, overall_prophet_forecast], 
        aics
    )

    print("Forecast Method Value Lengths:")
    print(f"  ARIMA: {len(overall_arima_forecast)}, SARIMA: {len(overall_sarima_forecast)}, "
        f"ExpSmoothing: {len(overall_exp_forecast)}, Prophet: {len(overall_prophet_forecast)}, "
        f"Ensemble: {len(overall_ensemble_forecast)}")

    
    # Store overall forecasts
    overall_forecasts = pd.DataFrame({
        'Date': forecast_dates,
        'ARIMA': overall_arima_forecast.values,
        'SARIMA': overall_sarima_forecast.values,
        'ExpSmoothing': overall_exp_forecast.values,
        'Prophet': overall_prophet_forecast.values,
        'Ensemble': overall_ensemble_forecast.values,
        'Level': 'Overall',
        'Segment': 'Total'
    })
    
    print(f"Overall forecast range: {overall_ensemble_forecast.min():,.0f} - {overall_ensemble_forecast.max():,.0f}")
    
    return overall_forecasts

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 11, Finished, Available, Finished)

# Running Overall Quantity Forecasting

Execute the overall quantity forecasting using multiple methods and ensemble approach.

## Running Models and Generating Forecast Quantities

In [10]:
results = []

for i, (train_df, test_df) in enumerate(sliding_splits):
    print(f"\n🔁 Fold {i+1} | Train: {train_df['Date'].min().date()} to {train_df['Date'].max().date()} | "
      f"Test: {test_df['Date'].min().date()} to {test_df['Date'].max().date()}")
    
    forecast_dates = test_df['Date'].values
    
    # Prepare inputs
    overall_series = train_df.set_index('Date')['Quantity Invoiced']
    overall_exog = train_df.set_index('Date')[exog_vars]
    
    # Forecast using the same functions
    arima_fc, _, aic_arima = forecast_arima(overall_series, steps=len(forecast_dates))
    sarima_fc, _, aic_sarima = forecast_sarima(overall_series, exog=overall_exog, steps=len(forecast_dates))
    exp_fc, _, aic_exp = forecast_exponential_smoothing(overall_series, steps=len(forecast_dates))
    prophet_fc, _, aic_prophet = forecast_prophet(overall_series, steps=len(forecast_dates), exog=overall_exog)
    extra_fc, _, aic_extra = forecast_extra_trees(overall_series, steps=len(forecast_dates), exog=overall_exog)
    
    # Ensemble
    ensemble_fc = ensemble_forecast([arima_fc, sarima_fc, exp_fc, prophet_fc, extra_fc], [aic_arima, aic_sarima, aic_exp, aic_prophet, aic_extra])
    
    # True values
    y_true = test_df['Quantity Invoiced'].values
    
    # Individual MAPEs
    mape_arima = mean_absolute_percentage_error(y_true, arima_fc.values)
    mape_sarima = mean_absolute_percentage_error(y_true, sarima_fc.values)
    mape_exp = mean_absolute_percentage_error(y_true, exp_fc.values)
    mape_prophet = mean_absolute_percentage_error(y_true, prophet_fc.values)
    mape_extra = mean_absolute_percentage_error(y_true, extra_fc.values)
    mape_ensemble = mean_absolute_percentage_error(y_true, ensemble_fc.values)

    print(f"MAPE ARIMA: {mape_arima:.2%} | SARIMA: {mape_sarima:.2%} | Exp: {mape_exp:.2%} | Prophet: {mape_prophet:.2%} | ExtraTrees: {mape_extra:.2%} | Ensemble: {mape_ensemble:.2%}")
    
    # Store all results
    results.append({
        "Fold": i + 1,
        "TrainStart": train_df['Date'].min(),
        "TrainEnd": train_df['Date'].max(),
        "TestStart": test_df['Date'].min(),
        "TestEnd": test_df['Date'].max(),
        "MAPE_ARIMA": mape_arima,
        "MAPE_SARIMA": mape_sarima,
        "MAPE_ExpSmoothing": mape_exp,
        "MAPE_PROPHET": mape_prophet,
        "MAPE_ExtraTrees": mape_extra,
        "MAPE_Ensemble": mape_ensemble
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)


StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 12, Finished, Available, Finished)


🔁 Fold 1 | Train: 2015-01-01 to 2023-12-01 | Test: 2024-01-01 to 2024-03-01


22:21:14 - cmdstanpy - INFO - Chain [1] start processing
22:21:14 - cmdstanpy - INFO - Chain [1] done processing


Model weights - ARIMA: 0.012, SARIMA: 0.014, EXP: 0.014, Prophet: 0.179
MAPE ARIMA: 22.21% | SARIMA: 4.09% | Exp: 19.75% | Prophet: 23.65% | ExtraTrees: 14.74% | Ensemble: 16.33%

🔁 Fold 2 | Train: 2015-02-01 to 2024-01-01 | Test: 2024-02-01 to 2024-04-01


22:21:38 - cmdstanpy - INFO - Chain [1] start processing
22:21:38 - cmdstanpy - INFO - Chain [1] done processing


...

Model weights - ARIMA: 0.012, SARIMA: 0.014, EXP: 0.014, Prophet: 0.179
MAPE ARIMA: 10.29% | SARIMA: 7.97% | Exp: 13.51% | Prophet: 6.46% | ExtraTrees: 8.51% | Ensemble: 8.13%

🔁 Fold 9 | Train: 2015-09-01 to 2024-08-01 | Test: 2024-09-01 to 2024-11-01


22:24:23 - cmdstanpy - INFO - Chain [1] start processing
22:24:23 - cmdstanpy - INFO - Chain [1] done processing


Model weights - ARIMA: 0.012, SARIMA: 0.014, EXP: 0.014, Prophet: 0.179
MAPE ARIMA: 8.59% | SARIMA: 13.07% | Exp: 12.94% | Prophet: 9.12% | ExtraTrees: 11.93% | Ensemble: 11.41%

🔁 Fold 10 | Train: 2015-10-01 to 2024-09-01 | Test: 2024-10-01 to 2024-12-01


22:24:48 - cmdstanpy - INFO - Chain [1] start processing
22:24:48 - cmdstanpy - INFO - Chain [1] done processing


Model weights - ARIMA: 0.012, SARIMA: 0.014, EXP: 0.014, Prophet: 0.179
MAPE ARIMA: 22.25% | SARIMA: 7.52% | Exp: 8.69% | Prophet: 9.90% | ExtraTrees: 14.28% | Ensemble: 13.41%


In [11]:
display(arima_fc)

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 13, Finished, Available, Finished)

2024-10-01    2.513722e+07
2024-11-01    2.498452e+07
2024-12-01    2.497574e+07
Freq: MS, Name: predicted_mean, dtype: float64

In [12]:
# ensure the output window is wide enough to prevent wrapping
pd.set_option('display.width', 250)
pd.set_option('display.max_columns', None)

results_df = pd.DataFrame(results)
print("\n📊 Sliding Window Validation Summary:")
print(results_df)

print(f"\n✅ Average Ensemble MAPE across {len(results_df)} folds: {results_df['MAPE_Ensemble'].mean():.2%}")

StatementMeta(, 801c3e88-6099-4f49-8b5f-e17e0aa57667, 14, Finished, Available, Finished)


📊 Sliding Window Validation Summary:
   Fold TrainStart   TrainEnd  TestStart    TestEnd  MAPE_ARIMA  MAPE_SARIMA  MAPE_ExpSmoothing  MAPE_PROPHET  MAPE_ExtraTrees  MAPE_Ensemble
0     1 2015-01-01 2023-12-01 2024-01-01 2024-03-01    0.222060     0.040854           0.197494      0.236476         0.147377       0.163320
1     2 2015-02-01 2024-01-01 2024-02-01 2024-04-01    0.134007     0.069291           0.173408      0.116294         0.057325       0.068657
2     3 2015-03-01 2024-02-01 2024-03-01 2024-05-01    0.096398     0.115639           0.081677      0.071808         0.084326       0.081704
3     4 2015-04-01 2024-03-01 2024-04-01 2024-06-01    0.092158     0.194939           0.140275      0.163802         0.155431       0.154915
4     5 2015-05-01 2024-04-01 2024-05-01 2024-07-01    0.147662     0.233025           0.169728      0.147191         0.171725       0.164947
5     6 2015-06-01 2024-05-01 2024-06-01 2024-08-01    0.142119     0.103866           0.161321      0.171132 